# HPT PyTorch Lightning With `spotPython` and CNNs on CIFAR10 {#sec-hyperparameter-tuning-for-pytorch-32}

In this tutorial, we will show how `spotPython` can be integrated into the `PyTorch` Lightning
training workflow for a classification task.

This chapter describes the hyperparameter tuning of a `PyTorch Lightning` network on the CIFAR10 data set.
It is based on the PyTorch Lightning tutorial "INCEPTION, RESNET AND DENSENET" by Phillip Lippe, see [https://lightning.ai/docs/pytorch/latest/notebooks/course_UvA-DL/04-inception-resnet-densenet.html](https://lightning.ai/docs/pytorch/latest/notebooks/course_UvA-DL/04-inception-resnet-densenet.html).



## Step 1: Setup {#sec-setup-33}

* Before we consider the detailed experimental setup, we select the parameters that affect run time, initial design size, etc. 
* The parameter `MAX_TIME` specifies the maximum run time in seconds.
* The parameter `INIT_SIZE` specifies the initial design size.
* The parameter `WORKERS` specifies the number of workers. 
* The prefix `PREFIX` is used for the experiment name and the name of the log file.


In [ ]:
MAX_TIME = 1
INIT_SIZE = 5
WORKERS = 0
PREFIX="32"

::: {.callout-caution}
### Caution: Run time and initial design size should be increased for real experiments

* `MAX_TIME` is set to one minute for demonstration purposes. For real experiments, this should be increased to at least 1 hour.
* `INIT_SIZE` is set to 5 for demonstration purposes. For real experiments, this should be increased to at least 10.
* `WORKERS` is set to 0 for demonstration purposes. For real experiments, this should be increased. See the warnings that are printed when the number of workers is set to 0.

:::

::: {.callout-note}
### Note: Device selection

* Although there are no .cuda() or .to(device) calls required, because Lightning does these for you, see
[LIGHTNINGMODULE](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html), we would like to know which device is used. Threrefore, we imitate the LightningModule behaviour which selects the highest device.
* The method `spotPython.utils.device.getDevice()` returns the device that is used by Lightning.
:::


## Step 2: Initialization of the `fun_control` Dictionary

`spotPython` uses a Python dictionary for storing the information required for the hyperparameter tuning process.

In [ ]:
from spotPython.utils.init import fun_control_init
from spotPython.utils.file import get_experiment_name, get_spot_tensorboard_path
from spotPython.utils.device import getDevice

experiment_name = get_experiment_name(prefix=PREFIX)
fun_control = fun_control_init(
    spot_tensorboard_path=get_spot_tensorboard_path(experiment_name),
    num_workers=WORKERS,
    device=getDevice(),
    _L_in=16 * 5 * 5,
    _L_out=10,
    TENSORBOARD_CLEAN=True)

## Step 3: PyTorch Data Loading {#sec-data-loading-31}

### Lightning Dataset and DataModule

The data loading and preprocessing is handled by `Lightning` and `PyTorch`.
Torchvision provides many built-in datasets in the `torchvision.datasets` module,
as well as utility classes for building your own datasets. All datasets are subclasses
of `torch.utils.data.Dataset`, i.e, they have `__getitem__` and `__len__`  methods implemented.
Hence, they can all be passed to a `torch.utils.data.DataLoader` which can load multiple
samples in parallel using `torch.multiprocessing` workers
We use the following classes:

*   `CIFAR10`: A class that loads the CIFAR10 data. [[LINK]](https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html#torchvision.datasets.CIFAR10)
*   `CIFAR10DataModule`: A class that prepares the data for training and testing. [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/cifar10/cifar10datamodule.py)

## Step 4: Preprocessing {#sec-preprocessing-31}

Preprocessing is handled by `Lightning` and `PyTorch`. It can be implemented in the `CIFAR10DataModule` class [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/ccifar10datamodule.py).

In [ ]:
from spotPython.light.cifar10.cifar10datamodule import CIFAR10DataModule

data_module = CIFAR10DataModule(batch_size=32, data_dir="./data", num_workers=WORKERS)


In [ ]:
data_module.prepare_data()

In [ ]:
data_module.setup()

In [ ]:

imgs, _ = next(iter(data_module.train_dataloader()))
print("Batch mean", imgs.mean(dim=[0, 2, 3]))
print("Batch std", imgs.std(dim=[0, 2, 3]))





## Step 5: Select the NN Model (`algorithm`) and `core_model_hyper_dict` {#sec-selection-of-the-algorithm-31}

`spotPython` includes the `NetLinearBase` class [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/netlinearbase.py) for configurable neural networks.
The class is imported here. It inherits from the class `Lightning.LightningModule`, which is the base class for all models in `Lightning`. `Lightning.LightningModule` is a subclass of `torch.nn.Module` and provides additional functionality for the training and testing of neural networks. The class `Lightning.LightningModule` is described in the [Lightning documentation](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html).

* Here we simply add the NN Model to the fun_control dictionary by calling the function `add_core_model_to_fun_control`:

In [ ]:
from spotPython.light.netlinearbase import NetLinearBase
from spotPython.data.light_hyper_dict import LightHyperDict
from spotPython.hyperparameters.values import add_core_model_to_fun_control
add_core_model_to_fun_control(core_model=NetLinearBase,
                              fun_control=fun_control,
                              hyper_dict= LightHyperDict)

The `NetLinearBase` is a configurable neural network. The hyperparameters of the model are specified in the `core_model_hyper_dict` dictionary [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/data/light_hyper_dict.json).

## Step 6: Modify `hyper_dict` Hyperparameters for the Selected Algorithm aka `core_model` {#sec-modification-of-hyperparameters-31}

 `spotPython` provides functions for modifying the hyperparameters, their bounds and factors as well as for activating and de-activating hyperparameters without re-compilation of the Python source code. These functions were described in @sec-modification-of-hyperparameters-14.

::: {.callout-caution}
### Caution: Small number of epochs for demonstration purposes

* `epochs` and `patience` are set to small values for demonstration purposes. These values are too small for a real application.
* More resonable values are, e.g.:
  * `modify_hyper_parameter_bounds(fun_control, "epochs", bounds=[7, 9])` and
  * `modify_hyper_parameter_bounds(fun_control, "patience", bounds=[2, 7])`
:::

In [ ]:
from spotPython.hyperparameters.values import modify_hyper_parameter_bounds

modify_hyper_parameter_bounds(fun_control, "l1", bounds=[5,8])
modify_hyper_parameter_bounds(fun_control, "epochs", bounds=[6,13])
modify_hyper_parameter_bounds(fun_control, "batch_size", bounds=[2, 8])

In [ ]:
from spotPython.hyperparameters.values import modify_hyper_parameter_levels
modify_hyper_parameter_levels(fun_control, "optimizer",["Adam", "AdamW", "Adamax", "NAdam"])
# modify_hyper_parameter_levels(fun_control, "optimizer", ["Adam"])

Now, the dictionary `fun_control` contains all information needed for the hyperparameter tuning. Before the hyperparameter tuning is started, it is recommended to take a look at the experimental design. The method `gen_design_table` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/utils/eda.py) generates a design table as follows:

In [ ]:
#| fig-cap: Experimental design for the hyperparameter tuning.
from spotPython.utils.eda import gen_design_table
print(gen_design_table(fun_control))

This allows to check if all information is available and if the information is correct.

::: {.callout-note}
### Note: Hyperparameters of the Tuned Model and the `fun_control` Dictionary
The updated `fun_control` dictionary can be shown with the command `fun_control["core_model_hyper_dict"]`.
:::


## Step 7: Data Splitting, the Objective (Loss) Function and the Metric

### Evaluation  {#sec-selection-of-target-function-31}

The evaluation procedure requires the specification of two elements:

1. the way how the data is split into a train and a test set (see @sec-data-splitting-14)
2. the loss function (and a metric).

::: {.callout-caution}
### Caution: Data Splitting in Lightning

* The data splitting is handled by `Lightning`.

:::

### Loss Functions and Metrics {#sec-loss-functions-and-metrics-31}

The loss function is specified in the configurable network class [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/netlinearbase.py)
We will use CrossEntropy loss for the multiclass-classification task.

### Metric {#sec-metric-31}

* We will use the default accuracy metric for the evaluation of the model.

Similar to the loss function, the metric is specified in the configurable network class [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/netlinearbase.py).

::: {.callout-caution}
### Caution: Loss Function and Metric in Lightning

* The loss function and the metric are not hyperparameters that can be tuned with `spotPython`.
* They are handled by `Lightning`.

:::

## Step 8: Calling the SPOT Function

### Preparing the SPOT Call {#sec-prepare-spot-call-31}

The following code passes the information about the parameter ranges and bounds to `spot`.
It extracts the variable types, names, and bounds

In [ ]:
from spotPython.hyperparameters.values import (get_bound_values,
    get_var_name,
    get_var_type,)
var_type = get_var_type(fun_control)
var_name = get_var_name(fun_control)
lower = get_bound_values(fun_control, "lower")
upper = get_bound_values(fun_control, "upper")

### The Objective Function `fun` {#sec-the-objective-function-31}

The objective function `fun` from the class `HyperLight` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/hyperlight.py) is selected next. It implements an interface from `PyTorch`'s training, validation, and testing methods to `spotPython`.

In [ ]:
from spotPython.fun.hyperlight import HyperLight
fun = HyperLight().fun

### Starting the Hyperparameter Tuning {#sec-call-the-hyperparameter-tuner-31}

The `spotPython` hyperparameter tuning is started by calling the `Spot` function [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/spot/spot.py) as described in @sec-call-the-hyperparameter-tuner-14.

In [ ]:
import numpy as np
from spotPython.spot import spot
from math import inf
spot_tuner = spot.Spot(fun=fun,
                   lower = lower,
                   upper = upper,
                   fun_evals = inf,
                   max_time = MAX_TIME,
                   tolerance_x = np.sqrt(np.spacing(1)),
                   var_type = var_type,
                   var_name = var_name,
                   show_progress= True,
                   fun_control = fun_control,
                   log_level=10,
                   design_control={"init_size": INIT_SIZE},
                   surrogate_control={"noise": True,
                                      "min_theta": -4,
                                      "max_theta": 3,
                                      "n_theta": len(var_name),
                                      "model_fun_evals": 10_000,
                                      })
spot_tuner.run()

## Step 9: Tensorboard {#sec-tensorboard-31}

The textual output shown in the console (or code cell) can be visualized with Tensorboard.

```{raw}
tensorboard --logdir="runs/"
```

Further information can be found in the [PyTorch Lightning documentation](https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.loggers.tensorboard.html) for Tensorboard.

## Step 10: Results {#sec-results-31}

After the hyperparameter tuning run is finished, the results can be analyzed as described in @sec-results-14.

In [ ]:
#| fig-cap: Progress plot. *Black* dots denote results from the initial design. *Red* dots  illustrate the improvement found by the surrogate model based optimization.
spot_tuner.plot_progress(log_y=False,
    filename="./figures/" + experiment_name+"_progress.png")

In [ ]:
#| fig-cap: Results of the hyperparameter tuning.
from spotPython.utils.eda import gen_design_table
print(gen_design_table(fun_control=fun_control, spot=spot_tuner))

In [ ]:
#| fig-cap: 'Variable importance plot, threshold 0.025.'
spot_tuner.plot_importance(threshold=0.025,
    filename="./figures/" + experiment_name+"_importance.png")

### Get the Tuned Architecture {#sec-get-spot-results-31}

In [ ]:
from spotPython.light.utils import get_tuned_architecture
config = get_tuned_architecture(spot_tuner, fun_control)

* Test on the full data set

In [ ]:
from spotPython.light.traintest import test_model
test_model(config, fun_control)

In [ ]:
from spotPython.light.traintest import load_light_from_checkpoint

model_loaded = load_light_from_checkpoint(config, fun_control)

### Cross Validation With Lightning

* The `KFold` class from `sklearn.model_selection` is used to generate the folds for cross-validation.
* These mechanism is used to generate the folds for the final evaluation of the model.
* The `CrossValidationDataModule` class [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/crossvalidationdatamodule.py) is used to generate the folds for the hyperparameter tuning process.
* It is called from the `cv_model` function [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/traintest.py).

In [ ]:
from spotPython.light.traintest import cv_model
# set the number of folds to 10
fun_control["k_folds"] = 10
cv_model(config, fun_control)

::: {.callout-note}
### Note: Evaluation for the Final Comaprison

* This is the evaluation that will be used in the comparison.

:::


### Detailed Hyperparameter Plots

In [ ]:
#| fig-cap: Contour plots.
filename = "./figures/" + experiment_name
spot_tuner.plot_important_hyperparameter_contour(filename=filename)

### Parallel Coordinates Plot

In [ ]:
#| fig-cap: Parallel coordinates plots
spot_tuner.parallel_plot()

### Plot all Combinations of Hyperparameters

* Warning: this may take a while.

In [ ]:
PLOT_ALL = False
if PLOT_ALL:
    n = spot_tuner.k
    for i in range(n-1):
        for j in range(i+1, n):
            spot_tuner.plot_contour(i=i, j=j, min_z=min_z, max_z = max_z)

### Visualizing the Activation Distribution

::: {.callout-note}
### Reference:

* The following code is based on [[PyTorch Lightning TUTORIAL 2: ACTIVATION FUNCTIONS]](https://lightning.ai/docs/pytorch/stable/notebooks/course_UvA-DL/02-activation-functions.html), Author: Phillip Lippe, License: [[CC BY-SA]](https://creativecommons.org/licenses/by-sa/3.0/), Generated: 2023-03-15T09:52:39.179933.

:::

After we have trained the models, we can look at the actual activation values that find inside the model. For instance, how many neurons are set to zero in ReLU? Where do we find most values in Tanh? To answer these questions, we can write a simple function which takes a trained model, applies it to a batch of images, and plots the histogram of the activations inside the network:

In [ ]:
from spotPython.torch.activation import Sigmoid, Tanh, ReLU, LeakyReLU, ELU, Swish
act_fn_by_name = {"sigmoid": Sigmoid, "tanh": Tanh, "relu": ReLU, "leakyrelu": LeakyReLU, "elu": ELU, "swish": Swish}

In [ ]:
from spotPython.hyperparameters.values import get_one_config_from_X
X = spot_tuner.to_all_dim(spot_tuner.min_X.reshape(1,-1))
config = get_one_config_from_X(X, fun_control)
model = fun_control["core_model"](**config, _L_in=64, _L_out=11)
model

In [ ]:
from spotPython.utils.eda import visualize_activations
visualize_activations(model, color=f"C{0}")

## Appendix

### Differences to the spotPython Approaches for `torch`, `sklearn` and `river`

::: {.callout-caution}
#### Caution: Data Loading in Lightning

* Data loading is handled independently from the `fun_control` dictionary by `Lightning` and `PyTorch`.
* In contrast to `spotPython` with `torch`, `river` and `sklearn`, the data sets are not added to the `fun_control` dictionary.
:::

#### Specification of the Preprocessing Model {#sec-specification-of-preprocessing-model-31}

The `fun_control` dictionary, the `torch`, `sklearn`and `river` versions of `spotPython` allow the specification of a data preprocessing pipeline, e.g., for the scaling of the data or for the one-hot encoding of categorical variables, see @sec-specification-of-preprocessing-model-14. This feature is not used in the `Lightning` version.

:::{.callout-caution}
#### Caution: Data preprocessing in Lightning

Lightning allows the data preprocessing to be specified in the `LightningDataModule` class. It is not considered here, because it should be computed at one location only.
:::

### Taking a Look at the Data {#sec-taking-a-look-at-the-data-31}

In [ ]:
import torch
# from spotPython.light.csvdataset import CSVDataset
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Create an instance of CSVDataset
# dataset = CSVDataset(csv_file="./data/VBDP/train.csv", train=True)
dataset = CIFAR10(root=fun_control["DATASET_PATH"], train=True, download=True, transform=transform)
# show the first element of the input data
print(dataset[0][0])
# show the size of the dataset
print(f"Dataset Size: {len(dataset)}")

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Set batch size for DataLoader
batch_size = 3
# Create DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(dataloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
# Set batch size for DataLoader
batch_size = 3
# Create DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataiter = iter(dataloader)
images, labels = next(dataiter)


# # Iterate over the data in the DataLoader
# for batch in dataloader:
#     inputs, targets = batch
#     # print(f"Batch Size: {inputs.size(0)}")
#     # print("---------------")
#     # print(f"Inputs: {inputs}")
#     # print(f"Targets: {targets}")
#     break

# Tests

In [ ]:
from spotPython.light.cnn.googlenet import InceptionBlock
import torch
import torch.nn as nn
block = InceptionBlock(3,
            {"3x3": 32, "5x5": 16},
            {"1x1": 16, "3x3": 32, "5x5": 8, "max": 8},
            nn.ReLU)
x = torch.randn(1, 3, 32, 32)
y = block(x)
y.shape

In [ ]:
from spotPython.light.cnn.googlenet import GoogleNet
import torch
import torch.nn as nn
model = GoogleNet()
x = torch.randn(1, 3, 32, 32)
y = model(x)
y.shape

In [ ]:
from spotPython.light.cnn.netcnnbase import NetCNNBase
from spotPython.light.cnn.googlenet import GoogleNet
import torch
import torch.nn as nn
config = {"c_in": 3, "c_out": 10, "act_fn": nn.ReLU}
fun_control = {"core_model": GoogleNet}
model = NetCNNBase(config, fun_control)
x = torch.randn(1, 3, 32, 32)
y = model(x)
y.shape

In [ ]:
fun_control["core_model"].__name__

In [ ]:
from spotPython.light.trainmodel import train_model
config = {"c_in": 3, "c_out": 10, "act_fn": nn.ReLU, "optimizer_name": "Adam", "optimizer_hparams": {"lr": 1e-3, "weight_decay": 1e-4}}
fun_control = {"core_model": GoogleNet}
model, result = train_model(config, fun_control)
result

## Test for fun()

In [ ]:
MAX_TIME = 1
INIT_SIZE = 5
WORKERS = 0
PREFIX="32"
from spotPython.utils.init import fun_control_init
from spotPython.utils.file import get_experiment_name, get_spot_tensorboard_path
from spotPython.utils.device import getDevice

experiment_name = get_experiment_name(prefix=PREFIX)
fun_control = fun_control_init(
    spot_tensorboard_path=get_spot_tensorboard_path(experiment_name),
    num_workers=WORKERS,
    device=getDevice(),
    _L_in=16 * 5 * 5,
    _L_out=10,
    TENSORBOARD_CLEAN=True)

In [ ]:
from spotPython.light.cnn.googlenet import GoogleNet
from spotPython.data.lightning_hyper_dict import LightningHyperDict
from spotPython.hyperparameters.values import add_core_model_to_fun_control
add_core_model_to_fun_control(core_model=GoogleNet,
                              fun_control=fun_control,
                              hyper_dict= LightningHyperDict)

# Lightning With SPOT:

## 1. JSON


 `data.lightning_hyper_dict.json`

In [ ]:
  "GoogleNet":
    {
    "act_fn": {
            "levels": ["Sigmoid",
                       "Tanh",
                       "ReLU",
                       "LeakyReLU",
                       "ELU",
                       "Swish"],
            "type": "factor",
            "default": "ReLU",
            "transform": "None",
            "class_name": "spotPython.torch.activation",
            "core_model_parameter_type": "instance",
            "lower": 0,
            "upper": 5},
    "optimizer_name": {
        "levels": [
                   "Adam"
                ],
        "type": "factor",
        "default": "Adam",
        "transform": "None",
        "class_name": "torch.optim",
        "core_model_parameter_type": "str",
        "lower": 0,
        "upper": 0}
    }

## 2. Class HyperLightning.py

 ```{python}
 def fun(self,
        X: np.ndarray,
        fun_control: dict = None) -> np.ndarray:
```

1. Args: `numpy.array` from `spotPython`: hyperparameters as numerical values
2. Generates a dictionary with hyperparameters, e.g.:


   ```{JSON}
   config: {
      'act_fn': <class 'spotPython.torch.activation.ReLU'>,
      'optimizer_name': 'Adam'}`
   ```

3. Passes dictionary to method `train_model()` 

## 3. train_model()

 `def train_model(config: dict, fun_control: dict):`

1. Prepares the data, e.g., `CIFAR10DataModule`
2. Sets up the Trainer
3. Sets up the model, e.g.,

```{python}
model = NetCNNBase(
            model_name=fun_control["core_model"].__name__,
            model_hparams=config,
            optimizer_name="Adam",
            optimizer_hparams={"lr": 1e-3, "weight_decay": 1e-4},
        )
```

## 4. netcnnbase.py

```{python}
class NetCNNBase(L.LightningModule):
    def __init__(self,
                model_name,
                model_hparams,
                optimizer_name,
                optimizer_hparams):
```

1. Saves hyperparameters in `self.hparams`
2. Creates model
3. Creates loss module
4. Creates optimizer
5. Defines forward pass
6. Defines training step
7. Defines validation step
8. Defines test step

## 5. GoogleNet

```{python}
class GoogleNet(nn.Module):
    """GoogleNet architecture

    Args:
        num_classes (int):
            Number of classes for the classification task. Defaults to 10.
        act_fn_name (str):
            Name of the activation function. Defaults to "relu".
        **kwargs:
            Additional keyword arguments.

    Attributes:
        hparams (SimpleNamespace):
            Namespace containing the hyperparameters.
        input_net (nn.Sequential):
            Input network.
        inception_blocks (nn.Sequential):
            Inception blocks.
        output_net (nn.Sequential):
            Output network.

    Returns:
        (torch.Tensor):
            Output tensor of the GoogleNet architecture

    Examples:
        >>> from spotPython.light.cnn.googlenet import GoogleNet
            import torch
            import torch.nn as nn
            model = GoogleNet()
            x = torch.randn(1, 3, 32, 32)
            y = model(x)
            y.shape
            torch.Size([1, 10])
    """
```

## 6. InceptionBlock

```{python}
class InceptionBlock(nn.Module):
    def __init__(self, c_in, c_red: dict, c_out: dict, act_fn):
        """
        Inception block as used in GoogLeNet.

        Args:
            c_in:
                Number of input feature maps from the previous layers
            c_red:
                Dictionary with keys "3x3" and "5x5" specifying
                the output of the dimensionality reducing 1x1 convolutions
            c_out:
                Dictionary with keys "1x1", "3x3", "5x5", and "max"
            act_fn:
                Activation class constructor (e.g. nn.ReLU)

        Returns:
            torch.Tensor:
                Output tensor of the inception block

        Examples:
            >>> from spotPython.light.cnn.googlenet import InceptionBlock
                import torch
                import torch.nn as nn
                block = InceptionBlock(3,
                            {"3x3": 32, "5x5": 16},
                            {"1x1": 16, "3x3": 32, "5x5": 8, "max": 8},
                            nn.ReLU)
                x = torch.randn(1, 3, 32, 32)
                y = block(x)
                y.shape
                torch.Size([1, 64, 32, 32])

        """
```

# Sample Run

In [3]:
from spotPython.utils.init import fun_control_init
from spotPython.utils.file import get_experiment_name, get_spot_tensorboard_path
from spotPython.utils.device import getDevice
from spotPython.light.cnn.googlenet import GoogleNet
from spotPython.data.lightning_hyper_dict import LightningHyperDict
from spotPython.hyperparameters.values import add_core_model_to_fun_control
from spotPython.fun.hyperlightning import HyperLightning
from spotPython.hyperparameters.values import get_default_hyperparameters_as_array

MAX_TIME = 1
INIT_SIZE = 3
WORKERS = 8
PREFIX="TEST"
experiment_name = get_experiment_name(prefix=PREFIX)
fun_control = fun_control_init(
    spot_tensorboard_path=get_spot_tensorboard_path(experiment_name),
    num_workers=WORKERS,
    device=getDevice(),
    _L_in=3,
    _L_out=10,
    TENSORBOARD_CLEAN=True)

add_core_model_to_fun_control(core_model=GoogleNet,
                            fun_control=fun_control,
                            hyper_dict= LightningHyperDict)

X_start = get_default_hyperparameters_as_array(fun_control)

hyper_light = HyperLightning(seed=126, log_level=50)
hyper_light.fun(X=X_start, fun_control=fun_control)

Global seed set to 42



core_model: <class 'spotPython.light.cnn.googlenet.GoogleNet'>
config: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
fun: Calling train_model
train_model: Starting
train_model: config: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
Files already downloaded and verified
Data mean [0.49139968 0.48215841 0.44653091]
Data std [0.24703223 0.24348513 0.26158784]
Files already downloaded and verified
Files already downloaded and verified


Global seed set to 42
Global seed set to 42


Files already downloaded and verified


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
Missing logger folder: runs/saved_models/GoogleNet/lightning_logs


...

model_hparams: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
self.hparams: "model_hparams":     {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
"model_name":        GoogleNet
"optimizer_hparams": {'lr': 0.001, 'weight_decay': 0.0001}
"optimizer_name":    Adam
create_model: Starting
model_name: GoogleNet
model_hparams: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
self.model: GoogleNet(
  (input_net): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (inception_blocks): Sequential(
    (0): InceptionBlock(
      (conv_1x1): Sequential(
        (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv_3x3): Sequent

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

fun: train_model returned


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
fun_control['core_model']
